# Setting up a Ray cluster with SmartSim

## 1. Start the cluster
We set up a SmartSim experiment, which will handle the launch of the Ray cluster.

First we import the relevant modules.

In [1]:
from smartsim import Experiment
from smartsim.ray import RayCluster

NUM_WORKERS = 8
alloc=None;#slurm.get_allocation(nodes=1+NUM_WORKERS, time="12:00:00", options={"ntasks": str(1+NUM_WORKERS), "partition": "spider", "C": "V100"})

In [2]:
exp = Experiment("ray-cluster", launcher='pbs')
cluster = RayCluster(name="ray-cluster", run_args={}, path='',
                     launcher='pbs', workers=NUM_WORKERS, alloc=alloc, batch=True, ray_num_cpus=38)

if cluster.batch:
    cluster.head_model.batch_settings._preamble += ["source ~/.bashrc", "conda activate smartsim"]
    if NUM_WORKERS:
        cluster.worker_model.batch_settings._preamble += ["source ~/.bashrc", "conda activate smartsim"]

exp.generate(cluster, overwrite=True)

16:00:50 crystal SmartSim[9377] INFO Working in previously created experiment


In [3]:
exp.start(cluster, block=False, summary=False)

16:00:53 crystal SmartSim[9377] ERROR An error occurred when launching head 
Check error and output files for details.
Name: head
Type: RayHead
Executable: /lus/scratch/arigazzi/anaconda3/envs/smartsim/bin/python
Executable arguments: ['/lus/snx11242/arigazzi/smartsim-dev/SmartSim/smartsim/ray/rayserverstarter.py', '--num-cpus=38', '--port=6780', '--redis-password=a8c08c1b-44fd-45c4-a7e6-d6756783a222']
Run Command: aprun

16:00:53 crystal SmartSim[9377] ERROR Job step head failed to launch


SmartSimError: Job step head failed to launch

## 2. Start the ray driver script

In [ ]:
cluster.start_ray_job('/lus/scratch/arigazzi/smartsim-dev/SmartSim/tutorials/05_starting_ray/templates/ppo_tune.py')

In [ ]:
cluster.start_ray_job('/lus/scratch/arigazzi/smartsim-dev/SmartSim/tutorials/05_starting_ray/templates/ppo_train.py')

In [ ]:
cluster.start_ray_job('/lus/scratch/arigazzi/smartsim-dev/SmartSim/tutorials/05_starting_ray/templates/mnist_pytorch_trainable.py')

## 3. Stop cluster and release allocation

In [ ]:
if alloc:
    slurm.release_allocation(alloc)

In [4]:
import ray
from ray import tune
import ray.util
import time
import numpy as np
import argparse
import os
from ray.tune.progress_reporter import JupyterNotebookReporter

ray.util.connect(cluster.head_model.address +":10001")
reporter = JupyterNotebookReporter(overwrite=True)

print("initialized")
tune.run(
    "PPO",
    stop={"episode_reward_max": 200},
    config={
        "framework": "torch",
        "env": "CartPole-v0",
        "num_gpus": 0,
        "lr": tune.grid_search(np.linspace (0.001, 0.01, 100).tolist()),
        "log_level": "ERROR",
        "num_cpus_per_worker": 1,
        "num_cpus_for_driver": 1,
    },
    local_dir="/lus/scratch/arigazzi/ray_local/",
    verbose=2,
    fail_fast=True,
    progress_reporter = reporter,
    log_to_file=True,
)

initialized
<IPython.core.display.HTML object>


(pid=32602) 2021-05-19 15:38:52,005	INFO trainer.py:694 -- Current log_level is ERROR. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


<IPython.core.display.HTML object>


(pid=32598) 2021-05-19 15:38:57,975	INFO trainer.py:694 -- Current log_level is ERROR. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=32600) 2021-05-19 15:38:57,981	INFO trainer.py:694 -- Current log_level is ERROR. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=32595) 2021-05-19 15:38:58,191	INFO trainer.py:694 -- Current log_level is ERROR. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=32597) 2021-05-19 15:38:58,188	INFO trainer.py:694 -- Current log_level is ERROR. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=32594) 2021-05-19 15:38:58,229	INFO trainer.py:694 -- Current log_level is ERROR. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=32602) 2021-05-19 15:39:24,282	INFO trainable.py:101 -- Trainable.setup took 32.277 seconds. If your trainable is slow to initialize, c

(pid=32576) Trial PPO_CartPole-v0_359a9_00000 reported episode_reward_max=86.0,episode_reward_min=9.0,episode_reward_mean=22.982758620689655,episode_len_mean=22.982758620689655,episode_media={},episodes_this_iter=174,policy_reward_min={},policy_reward_max={},policy_reward_mean={},custom_metrics={},sampler_perf={'mean_raw_obs_processing_ms': 1.6501029991226115, 'mean_inference_ms': 17.685853087411164, 'mean_action_processing_ms': 0.6844336755076307, 'mean_env_wait_ms': 0.8950426797003957, 'mean_env_render_ms': 0.0},off_policy_estimator={},num_healthy_workers=2,agent_timesteps_total=4000,timers={'sample_time_ms': 43255.553, 'sample_throughput': 92.474, 'learn_time_ms': 64934.756, 'learn_throughput': 61.6, 'update_time_ms': 7.278},info={'learner': defaultdict(<class 'dict'>, {'default_policy': {'learner_stats': {'allreduce_latency': 0.0, 'cur_kl_coeff': 0.2, 'cur_lr': 0.001, 'total_loss': 83.75345146656036, 'policy_loss': -0.05583573368494399, 'vf_loss': 83.80137264728546, 'vf_explained_v

...

(pid=32576) Trial PPO_CartPole-v0_359a9_00029 reported episode_reward_max=62.0,episode_reward_min=9.0,episode_reward_mean=21.03157894736842,episode_len_mean=21.03157894736842,episode_media={},episodes_this_iter=190,policy_reward_min={},policy_reward_max={},policy_reward_mean={},custom_metrics={},sampler_perf={'mean_raw_obs_processing_ms': 1.619285963872151, 'mean_inference_ms': 18.06243403806576, 'mean_action_processing_ms': 0.554287521732346, 'mean_env_wait_ms': 1.0460789449029497, 'mean_env_render_ms': 0.0},off_policy_estimator={},num_healthy_workers=2,agent_timesteps_total=4000,timers={'sample_time_ms': 44407.538, 'sample_throughput': 90.075, 'learn_time_ms': 122005.047, 'learn_throughput': 32.786, 'update_time_ms': 9.597},info={'learner': defaultdict(<class 'dict'>, {'default_policy': {'learner_stats': {'allreduce_latency': 0.0, 'cur_kl_coeff': 0.2, 'cur_lr': 0.003636363636363637, 'total_loss': 68.67967522144318, 'policy_loss': -0.05107551920809783, 'vf_loss': 68.72348308563232, 'v

(pid=53737, ip=172.30.49.190) 2021-05-19 15:58:25,799	INFO trainer.py:694 -- Current log_level is ERROR. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=53739, ip=172.30.49.190) 2021-05-19 15:58:25,805	INFO trainer.py:694 -- Current log_level is ERROR. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=53737, ip=172.30.49.190) 2021-05-19 15:58:41,733	INFO trainable.py:101 -- Trainable.setup took 15.935 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
(pid=53739, ip=172.30.49.190) 2021-05-19 15:58:41,721	INFO trainable.py:101 -- Trainable.setup took 15.916 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


(pid=32576) Trial PPO_CartPole-v0_359a9_00037 reported episode_reward_max=93.0,episode_reward_min=9.0,episode_reward_mean=22.725714285714286,episode_len_mean=22.725714285714286,episode_media={},episodes_this_iter=175,policy_reward_min={},policy_reward_max={},policy_reward_mean={},custom_metrics={},sampler_perf={'mean_raw_obs_processing_ms': 0.748811364897298, 'mean_inference_ms': 10.81839937158474, 'mean_action_processing_ms': 0.3127144585927598, 'mean_env_wait_ms': 0.4644452052342097, 'mean_env_render_ms': 0.0},off_policy_estimator={},num_healthy_workers=2,agent_timesteps_total=4000,timers={'sample_time_ms': 25902.238, 'sample_throughput': 154.427, 'learn_time_ms': 88348.739, 'learn_throughput': 45.275, 'update_time_ms': 7.504},info={'learner': defaultdict(<class 'dict'>, {'default_policy': {'learner_stats': {'allreduce_latency': 0.0, 'cur_kl_coeff': 0.2, 'cur_lr': 0.004363636363636364, 'total_loss': 89.68724203109741, 'policy_loss': -0.04823113637394272, 'vf_loss': 89.72783136367798,

(pid=49698) 2021-05-19 16:00:01,892	INFO trainer.py:694 -- Current log_level is ERROR. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=55388, ip=172.30.49.190) 2021-05-19 16:00:20,464	INFO trainer.py:694 -- Current log_level is ERROR. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=55395, ip=172.30.49.190) 2021-05-19 16:00:20,462	INFO trainer.py:694 -- Current log_level is ERROR. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=49751) 2021-05-19 16:00:22,574	INFO trainer.py:694 -- Current log_level is ERROR. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


KeyboardInterrupt: 

In [5]:
exp.stop(cluster)

16:00:30 crystal SmartSim[3737] INFO Stopping model workers with job name workers-CBHIFCQZY0IL
16:00:30 crystal SmartSim[3737] INFO Stopping model head with job name head-CBHIF67HSKFD
